# 1고지

In [1]:
import numpy as np

In [69]:
class Variable: # 객체 선언
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray) or isinstance(data, np.float64): # 타입 비교
                raise TypeError(f'{type(data)} is not supported.') # Error 표시 및 코드 종료

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func): # self 객체에 사용되는 함수 객체를 설정하는 메서드
        self.creator = func 

    #### 재귀 방식과 반복문 방식의 경우 메모리 할당 측면에서 반복문 방식이 유리하다.
    ### 재귀 방식은 호출 시마다 메모리에 누적되나 반복문의 경우 pop을 활용해 메모리가 누적되지 않은 상태로 작업을 수월한다.

    ## 1. 재귀 방식의 역전파
    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input
    #         x.grad = f.backward(self.grad)
    #         x.backward() # 뒤의 인자를 받아 backward 수행 및 새로나온 결과를 다시 뒤의 인자로 활용해 재귀하는 형태

    ## 2. 반복문을 이용한 역전파
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs: # 루프를 통한 역전파 수행
            f = funcs.pop() # 메모리 문제를 해결하기 위한 pop
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None: # 최초 노드로 진입할 경우 creator가 존재하지 않으므로 더이상의 수행이 진행되지 않음
                funcs.append(x.creator)

In [3]:
class Function:
    def __call__(self, input): # 객체를 함수처럼 호출하는 함수 객체로써 역할을 수행하기 위해 사용되는 매직 메서드
        x = input.data
        y = self.forward(x) # input이라는 variable 객체를 받아 순전파 수행 및 결과 저장

        output = Variable(as_array(y)) # 값이 ndarray 타입이 아닐 경우 변환해주고 variable로 감싸주는 형태
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    # 기반 클래스로 사용되므로 파생 클래스가 필수적으로 포함해야 하는 메서드인 forward와 backward를 설정하되
    # raise로 에러를 주어서 만약 객체에 아래 메서드가 포함되지 않을 경우 Function의 forward, backward 메서드가
    # 호출되므로 오류가 발생하고 코드가 종료되는 형태
    def forward(self, x):
        raise NotImplementedError() # 파생 클래스에서 구현되지 않았음을 알리는 에러

    def backward(self, gy):
        raise NotImplementedError()

In [4]:
def as_array(x): # 타입체크 및 ndarray 형태로 변환해주는 함수
    if np.isscalar(x):
        return np.array(x)
    return x

In [5]:
class Square(Function): # Function이라는 기반 클래스로부터 상속받는 Square 파생 클래스
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [6]:
class Exp(Function): # Function이라는 기반 클래스로부터 상속받는 Exp 파생 클래스
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gy = np.exp(x) * gy
        return gy

In [7]:
def numerical_diff(f, x, eps=1e-4): # 수치 미분을 수행하는 전역 함수
    if type(x.data - eps) is np.float64: # 타입 확인 및 형변환 1
        temp1 = np.array(x.data - eps)
        x0 = Variable(temp1)
    else:
        x0 = Variable(x.data - eps)
    if type(x.data + eps) is np.float64: # 타입 확인 및 형변환 2
        temp2 = np.array(x.data + eps)
        x1 = Variable(temp2)
    else:
        x1 = Variable(x.data + eps)
    
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2*eps)

In [8]:
def f(x): # 함수 테스트
    A = Square() # __call__ 메서드를 통해 객체를 그대로 생성
    B = Exp() # <__main__.Exp at 0x16a0a3257c0>
    C = Square() # <__main__.Square at 0x16a0afa3b80>
    return C(B(A(x)))

In [9]:
x = Variable(np.array(10.0))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100.0


In [10]:
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [11]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 검증
# assert 뒤의 값이 True가 아닐 경우, AssertionError 발생
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [12]:
# 역전파 1 (내부의 멤버변수를 다루지 않은 상태에서 backprop 수행)
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [13]:
# 역전파 2 (내부의 멤버변수를 차용해 backprop 수행)
y.grad = np.array(1.0)

C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

In [14]:
print(x.grad)

3.297442541400256


In [15]:
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [16]:
# 함수 객체를 활용해 실제 수학적 연산을 수행하듯이 함수를 정의
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

def exp(x):
    f = Exp()
    return f(x)
    # return Exp()(x)

In [17]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # Sqaure, Exp, Sqaure 3개의 함수 객체를 혼용하여 순전파 수행 (e**(x**2))**2 형태가 됨
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [18]:
x = Variable(np.array(0.5))
x = Variable(None)

x = Variable(1.0) # float에 대해서 처리가 되어있지 않으므로 오류 발생 (np.float의 경우 처리됨)

TypeError: <class 'float'> is not supported.

In [23]:
# 샘플 테스트 수행
# unittest라는 모듈을 받아서 시험하고자 하는 값을 넣고
# def 내부에 test로 시작하는 함수를 선언하면 unittest가 수행되고
# 이 함수 내부에서 assertEqual과 같은 unittest 모듈의 다른 메서드가
# 실행되면서 제대로 동작하는지 확인을 진행
# ipynb파일과 다르게 .py파일을 바로 사용해야 하는 경우에는 unittest를 통한
# 테스트 검증이 필요하고 만약 이미 release된 파일을 업데이트 해야 하는 경우에는
# test가 필요한데 큰 파일을 그대로 가져다 사용할 수 없기 때문에 이와 같은
# 테스트 방식을 수행합니다.
import unittest

class SquareTest(unittest.TestCase): # unittest.TestCase 클래스를 상속받는 파생클래스 생성
    # Case 1
    def test_forward(self): # unittest를 상속받는 클래스는 필수적으로 def에 들어가는 함수 이름에 test를 넣어주어야 한다.
        x = Variable(np.array(2.))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected) # assertEqual 함수를 unittest.TestCase로부터 받아서 들어오는 인자값끼리 값을 비교한다.
    
    # Case 2 (include Case 1)
    def test_backward(self):
        x = Variable(np.array(3.))
        y = square(x)
        y.backward()
        expected = np.array(6.)
        self.assertEqual(x.grad, expected)

    # Case 3 (include Case 1, 2)
    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # 무작위 입력값 주입
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

# np.allclose(a, b, rtol = 1e-05, atol = 1e-08)
# https://numpy.org/doc/stable/reference/generated/numpy.allclose.html
# 들어오는 a와 b 값의 오차를 측정하는 함수로 |a - b| <= (atol + rtol*|b|)
# 구문을 만족할 경우 True 반환, 나머지의 경우 False 반환

In [20]:
#########################################################
# test_forward(self) 함수만 선언하고 사용했을 때 결과 확인 #
#########################################################

# Case 1
unittest.main(argv=[''], verbosity=2, exit=False)

# .py 파일내에서 unittest.TestCase를 상속받는 클래스를 만든 경우에는
# 테스트 시 (첫 번째 경우)
# python -m unittest 파일명.py
# 라는 명령어를 터미널에 주어야 아래와 같은 실행 결과를 확인할 수 있음

# 또 다른 테스트 시 (두 번째 경우)
# 함수 본문 마지막에 unittest.main()을 추가한 뒤
# 정상적으로 python 파일명.py
# 를 터미널에서 실행해도 됨

test_forward (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [22]:
# Case 2
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [24]:
# Case 3
unittest.main(argv=[''], verbosity=2, exit=False)

test_backward (__main__.SquareTest) ... ok
test_forward (__main__.SquareTest) ... ok
test_gradient_check (__main__.SquareTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.021s

OK


In [25]:
# 터미널에서 .py에 있는 tests 디렉토리 내에 있는 테스트 파일들을 한꺼번에 검사하기 위해서는
# python -m unittest discover tests
# 라는 명령어를 주게 되면 test 폴더 내부에 있는 모든 test 파일들에 대해 검사를 수행한다.
# 여기서 discover라는 하위 명령은 discover 뒤에 지정된 디렉터리에 대해 파일이 있는지 탐색하는 명령어.

# 2고지

In [66]:
class Function:
    def __call__(self, inputs): # 제 1고지와 다르게 inputs로 값을 받아서 리스트 데이터 처리(list comprehension)
        xs = [x.data for x in inputs]
        ys = self.forward(xs) # inputs는 Variable 객체의 리스트이므로 리스트를 분리해서 data 추출

        outputs = [Variable(as_array(y)) for y in ys] # 순전파된 결과를 array로 변환한 다음 Variable의 인자값으로 주어 리스트에 다시 저장
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, x):
        raise NotImplementedError() # 파생 클래스에서 구현되지 않았음을 알리는 에러

    def backward(self, gy):
        raise NotImplementedError()

In [67]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

    def backward(self, gy):
        return gy, gy

In [70]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [30]:
class Function:
    def __call__(self, *inputs): # 제 2고지와 첫 Function과 다르게 inputs 값을 가변인자 형태로 받음
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # inputs는 Variable 객체의 리스트이므로 리스트를 분리해서 data 추출
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # 순전파된 결과를 array로 변환한 다음 Variable의 인자값으로 주어 리스트에 다시 저장
        
        self.generation = max([x.generation for x in inputs]) # 들어오는 값들의 generation값을 체크한 뒤 가장 높을 값을 현재 generation으로 설정
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError() # 파생 클래스에서 구현되지 않았음을 알리는 에러

    def backward(self, gy):
        raise NotImplementedError()

In [31]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y # 기반 클래스에서 isinstance 여부를 확인하여 tuple로 변환 처리됨

    def backward(self, gy):
        return gy, gy

In [32]:
def add(x0, x1): # 함수 객체 호출을 간편하게 하기 위해서 add 함수 정의
    return Add()(x0, x1)

In [34]:
class Variable: # 제 1고지에 있던 Variable 클래스를 수정(업데이트)
    def __init__(self, data):
        if data is not None:
            if isinstance(data, list):
                pass
            elif not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')
            

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self): # 한 번 사용했던 변수를 재사용하는 경우에 grad 값이 이미 존재하는 경우 충돌이 발생하므로 이를 clear해주는 메서드를 정의
        self.grad = None

    def set_creator(self, func): 
        self.creator = func
        # generation 추가. Node가 같은 층에서 여러 개가 들어갈 경우 역전파 수행 시 직렬로 수행되므로 한 층에 있는 다른 노드에 대해서 처리가 병렬적으로 되지 않기 때문에 우선순위를 매기는 generation 변수를 설정함
        self.generation = func.generation + 1

    def backward(self): 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) # 가변인자로 값을 받아 역전파를 수행하고
            if not isinstance(gxs, tuple): # 타입을 체크한 뒤
                gxs = (gxs,)
            for x, gx in zip(f.inputs, gxs): # zip 함수를 활용해 값을 체크하되
                if x.grad is None:
                    x.grad = gx
                else: # x + x = 2x와 같이 x값을 두 번 사용하는 경우 값이 이미 연산되어 들어가므로 이에 대해서 예외 처리를 진행 & += 연산자를 사용하게 될 경우 메모리 주소를 공유하게 되므로 메모리 주소를 다르게 사용하기 위해서 x.grad = x.grad + gx 구문을 사용함
                    x.grad = x.grad + gx
                if x.creator is not None:
                    funcs.append(x.creator)

In [35]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


In [36]:
x = Variable(np.array(3.))
y = add(x, x)
y.backward()
print(x.grad)
x.cleargrad()
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


In [153]:
# weakref 모듈 임포트
# 현재 순전파 및 역전파 코드는 서로가 서로를 참조하는 순환 참조 형태의 코드이기 때문에
# weakref 모듈을 사용해서 메모리 참조 상에서 누수 문제를 방지
import weakref

class Function:
    def __call__(self, *inputs): # 1고지와 다르게 파라미터 값을 가변 인자로 받음
        inputs = [as_variable(x) for x in inputs] # 그에 따라서 inputs 값을 list comprehension을 사용하여 변환
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple): # isinstance 함수를 사용하여 값의 타입 체크
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop: # Config 클래스의 클래스 변수를 바로 체크
            self.generation = max([x.generation for x in inputs]) # 세대 체크 및 세대값 대입
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs] # weakref.ref를 사용해 객체 내부에 output값 넣기
        
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [154]:
def as_variable(obj): # 타입 체크
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [155]:
class Config: # Flag 설정 class
    enable_backprop = True

In [157]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if isinstance(data, list):
                pass
            elif not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')
            

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    # 재귀 방식의 역전파
    # def backward(self):
    #     f = self.creator
    #     if f is not None:
    #         x = f.input
    #         x.grad = f.backward(self.grad)
    #         x.backward()

    # 반복문을 이용한 역전파
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [202]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [203]:
# 가변인자를 parameter로 받아서 계산
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

f = Add()

y = f(x0, x1)
print(y.data)

5


In [159]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [160]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

y = add(x0, x1)
print(y.data)

5


In [195]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, *xs):
        if isinstance(xs, tuple):
            xs = xs[0]
        forward_list = [x.data for x in xs]
        if len(forward_list) > 1:
            y = forward_list[0] + forward_list[1]
        else:
            raise TypeError(f"'Length : {len(forward_list)}' isn't worked in Function (That's not correct length).")
        return (y,)

    def backward(self, gy):
        return gy, gy

In [196]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
# xs = [Variable(np.array(2))] # 오류 테스트
f = Add()
ys = f(xs)
# y = ys[0]
print(ys.data)

5


In [225]:
# Variable work-space
class Variable:
    def __init__(self, data, name = None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported.')

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def cleargrad(self):
        self.grad = None

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    # decorator를 활용하여 Variable 클래스의 data 객체는 numpy 타입이므로 해당 타입의 기능들을 Variable 객체의 기능처럼 가져옴 (shape, ndim, size, dtype).
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self): # ndarray의 길이 측정용 매직메서드
        return len(self.data) 

    def __repr__(self): # print문으로 객체 호출 시 반환할 값을 정하는 매직메서드
        if self.data is None:
            return "Variable(None)"
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return f'variable({p}) from __repr__' # print 함수로 Variable 호출 시 Variable.data의 값을 리턴하도록 설정

    def __mul__(self, other): # 곱셈기능 지원
        return mul(self, other)

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
        
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None

In [226]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

In [227]:
def square(x):
    f = Square()
    return f(x)
    # return Square()(x)

In [228]:
class Add(Function): # Function 클래스를 상속받는 Add 클래스 선언
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [229]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

In [230]:
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [231]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(y.grad)

None


In [232]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

x.cleargrad() # 메모리 절약을 위해 변수를 재사용하는 경우 grad값이 남아있는 상태이므로 초기화 수행
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


In [234]:
x = Variable(np.array(2.0))         
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [235]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [243]:
# 메모리 할당 확인을 위한 모듈 설치

# !pip install memory_profiler
# %load_ext memory_profiler
# %memit

In [244]:
Config.enable_backprop = True
x = Variable(np.ones((100, 100, 100)))
y = square(square(square(x)))
y.backward()
print(x0.grad, x1.grad)
# %memit # peak memory: 138.91 MiB, increment: 0.00 MiB

2.0 1.0


In [245]:
# Config.enable_backprop = False
# x = Variable2(np.ones((100, 100, 100)))
# y = square(square(square(x)))
# # y.backward() # False일 경우 backward 사용 불가

In [246]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    print(f"*** Mode = {value}")
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)
        print("*** Done")

In [247]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)
    print("y:", y.data)
    # %memit # peak memory: 100.75 MiB, increment: 0.00 MiB

*** Mode = False
y: 4.0
*** Done


In [248]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)
    # %memit # peak memory: 100.71 MiB, increment: 0.00 MiB

*** Mode = False
*** Done


#### contextlib 예제

In [249]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start')
    try:
        yield
    finally:
        print('done')

with config_test():
    print('process...')

start
process...
done


#### Mul Class

In [251]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return x1*gy, x0*gy

In [252]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

In [253]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))
c = Variable(np.array(1.))

y = add(mul(a, b), c)
y.backward()

In [254]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [255]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))

y = a * b
print(y)

variable(6.0) from __repr__


In [256]:
Variable.__mul__ = mul
Variable.__add__ = add
Variable.__rmul__ = mul
Variable.__radd__ = add

In [257]:
a = Variable(np.array(3.))
b = Variable(np.array(2.))
c = Variable(np.array(1.))

y = a * b + c
y.backward()

In [258]:
print(y)
print(a.grad)
print(b.grad)

variable(7.0) from __repr__
2.0
3.0


In [259]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [260]:
x = Variable(np.array(2.))
y = x + np.array(3.0)
print(y)

variable(5.0) from __repr__


In [261]:
x = Variable(np.array(2.))
y = x + 3.0
print(y)

variable(5.0) from __repr__


In [262]:
y = 2.0 * x + 1.0
print(y)

variable(5.0) from __repr__


#### 기타 함수 및 메서드 추가

In [265]:
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy

In [266]:
def neg(x):
    return Neg()(x)

In [268]:
Variable.__neg__ = neg

In [269]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

In [270]:
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

In [271]:
Variable.__sub__ = sub
Variable.__rsub__ = rsub

In [272]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [273]:
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

In [274]:
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [275]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

In [278]:
def pow(x, c):
    return Pow(c)(x)

In [279]:
Variable.__pow__ = pow

In [280]:
x = Variable(np.array(2.0))
y = x ** 0
print(y)

variable(1.0) from __repr__
